In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

weather = pd.read_csv('data/weather.csv')
pallet_history = pd.read_csv('data/Pallet_history_Gold_Spike.csv')
inbound_loads = pd.read_csv('data/inbound_loads.csv')
outbound_loads = pd.read_csv('data/outbound_laods.csv')
demand_Kwtrain_val = pd.read_csv('data/demand_kWtrain_val.csv')

In [11]:
print(pallet_history)

        Unnamed: 0  source_system_id  warehouse_facility_id  customer_code  \
0                0                11                     42           3610   
1                1                11                     42           3610   
2                2                11                     42           3610   
3                3                11                     42           3610   
4                4                11                     42           3610   
...            ...               ...                    ...            ...   
138725      138725                11                     42         126053   
138726      138726                11                     42         126053   
138727      138727                11                     42         126053   
138728      138728                11                     42         126053   
138729      138729                11                     42             12   

       from_location_code to_location_code     transaction_date